In [1]:
import pandas as pd

In [2]:
df_fusionné = pd.read_csv('df_fusionné.csv')

In [3]:
df_fusionné_binaire = df_fusionné.sample(frac=0.2)
df_fusionné_binaire.iloc[:, 0] = df_fusionné_binaire.iloc[:, 0].apply(lambda x: 0 if x == 'N' else 1)

In [4]:
# Count the occurrences of each class
counts = df_fusionné_binaire.iloc[:, 0].value_counts()

# Find the number of instances in the minority class
min_count = counts.min()

# Create balanced DataFrame through undersampling
df_balanced_under = pd.concat([
    df_fusionné_binaire[df_fusionné_binaire.iloc[:, 0] == 0].sample(min_count),
    df_fusionné_binaire[df_fusionné_binaire.iloc[:, 0] == 1].sample(min_count)
])

# Shuffle the DataFrame to mix the classes
df_balanced_under = df_balanced_under.sample(frac=1)


In [5]:
# Count occurrences of 0 and 1 in the first column
value_counts = df_balanced_under.iloc[:, 0].value_counts()

# Display the counts
print(value_counts)

ColumnName
0    7492
1    7492
Name: count, dtype: int64


In [6]:
df_balanced_under

,ColumnName,0,1,2,3,4,5,6,7,8,...,710,711,712,713,714,715,716,717,718,719
108516,0,0.469760,0.469683,0.469593,0.469509,0.469483,0.469470,0.469482,0.469490,0.469378,...,0.470895,0.470751,0.470628,0.470512,0.470403,0.470298,0.470182,0.470066,0.469955,0.469849
97741,1,0.505150,0.500557,0.505827,0.498937,0.501057,0.503490,0.501024,0.497329,0.497723,...,0.441083,0.443887,0.441658,0.437687,0.438659,0.441981,0.436740,0.434280,0.440623,0.440049
105662,1,0.467167,0.467535,0.467937,0.468361,0.468749,0.469139,0.469530,0.469931,0.470396,...,0.465069,0.465305,0.465484,0.465648,0.465829,0.466017,0.466208,0.466415,0.466618,0.466845
34489,1,0.421587,0.417330,0.393667,0.397666,0.384051,0.370780,0.369369,0.371229,0.362030,...,0.589039,0.581814,0.584979,0.591274,0.587373,0.578713,0.583787,0.580559,0.570062,0.577272
134699,1,0.499425,0.499512,0.499611,0.499720,0.499832,0.499948,0.500072,0.500201,0.500334,...,0.499285,0.499295,0.499289,0.499281,0.499281,0.499287,0.499295,0.499311,0.499329,0.499358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138184,0,0.465440,0.464723,0.463898,0.462998,0.462199,0.461396,0.460576,0.459728,0.458719,...,0.467421,0.467115,0.466955,0.466849,0.466695,0.466536,0.466426,0.466283,0.466172,0.465990
16921,1,0.520870,0.500283,0.499014,0.503580,0.501425,0.498133,0.498608,0.499639,0.496777,...,0.539522,0.538391,0.539823,0.541684,0.543194,0.544307,0.541761,0.536651,0.536499,0.537326
184371,0,0.501497,0.501420,0.501342,0.501264,0.501195,0.501129,0.501067,0.501005,0.500926,...,0.502277,0.502188,0.502111,0.502037,0.501964,0.501890,0.501812,0.501733,0.501654,0.501575
34053,1,0.542829,0.547924,0.555252,0.553511,0.558322,0.563257,0.564288,0.564232,0.567315,...,0.496135,0.498156,0.496896,0.495086,0.493815,0.493080,0.496873,0.505500,0.504136,0.494260


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

X = df_balanced_under.iloc[:, 1:]  
y = df_balanced_under.iloc[:, 0] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
print(y_train.unique())

[1 0]


In [9]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [11]:
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# Build the model
model = Sequential([
    (LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], 1))),
    Dropout(0.3),
    (LSTM(100, return_sequences=True)),
    Dropout(0.3),
    (LSTM(100)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# Define callbacks
Early_Stopper = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=5, mode="min")
Checkpoint_Model = tf.keras.callbacks.ModelCheckpoint(monitor="val_accuracy",
                                                      save_best_only=True,
                                                      save_weights_only=True,
                                                      filepath="./modelcheck.weights.h5")

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model
Conv1D_Model = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[Early_Stopper, Checkpoint_Model])

# Evaluate the model
evaluation = model.evaluate(X_test, y_test)


Epoch 1/100


2024-03-12 22:49:49.853036: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


375/375 ━━━━━━━━━━━━━━━━━━━━ 47s 121ms/step - accuracy: 0.5014 - loss: 0.6945 - val_accuracy: 0.4962 - val_loss: 0.6934
Epoch 2/100
375/375 ━━━━━━━━━━━━━━━━━━━━ 44s 117ms/step - accuracy: 0.5139 - loss: 0.6930 - val_accuracy: 0.5155 - val_loss: 0.6925
Epoch 3/100
375/375 ━━━━━━━━━━━━━━━━━━━━ 44s 116ms/step - accuracy: 0.5054 - loss: 0.6936 - val_accuracy: 0.4908 - val_loss: 0.6929
Epoch 4/100
375/375 ━━━━━━━━━━━━━━━━━━━━ 44s 117ms/step - accuracy: 0.5085 - loss: 0.6931 - val_accuracy: 0.5576 - val_loss: 0.6915
Epoch 5/100
375/375 ━━━━━━━━━━━━━━━━━━━━ 44s 116ms/step - accuracy: 0.5351 - loss: 0.6917 - val_accuracy: 0.5716 - val_loss: 0.6869
Epoch 6/100
375/375 ━━━━━━━━━━━━━━━━━━━━ 44s 117ms/step - accuracy: 0.5318 - loss: 0.6954 - val_accuracy: 0.5038 - val_loss: 0.6931
Epoch 7/100
375/375 ━━━━━━━━━━━━━━━━━━━━ 44s 117ms/step - accuracy: 0.5076 - loss: 0.6943 - val_accuracy: 0.5038 - val_loss: 0.6933
Epoch 8/100
242/375 ━━━━━━━━━━━━━━━━━━━━ 14s 108ms/step - accuracy: 0.4995 - loss: 0.693

KeyboardInterrupt: 

In [ ]:
# Assuming your model is named 'Model'
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype("int32")  # Convert probabilities to binary labels

96/96 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step


In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, log_loss

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Precision
precision = precision_score(y_test, y_pred)
print(f"Precision: {precision:.2f}")

# Recall
recall = recall_score(y_test, y_pred)
print(f"Recall: {recall:.2f}")

# F1 Score
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")

# ROC-AUC Score
roc_auc = roc_auc_score(y_test, y_pred_probs)
print(f"ROC AUC Score: {roc_auc:.2f}")

# Now, you can calculate the log loss
logloss = log_loss(y_test, y_pred_probs)
print(f"Log Loss: {logloss:.2f}")


Confusion Matrix:
 [[ 560 1019]
 [ 292 1185]]
Precision: 0.54
Recall: 0.80
F1 Score: 0.64
ROC AUC Score: 0.63
Log Loss: 0.68
